In [ ]:
from tifffile import imread, imsave
import os, re, sys, csv, cv2

import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams

import napari
import copy
from skimage.segmentation import find_boundaries

import phenograph

sys.path.append('~/3D_IMC_paper/Python/python_3d_imc_tools')
from io_files import image_filepath_for_3D_stack

import random

from napari.utils.colormaps.colormaps import Colormap
from napari.utils.colormaps.standardize_color import transform_color
from napari.utils.colormaps.colormaps import color_dict_to_colormap

In [ ]:
def load_channel_stack_for_napari(channel_name_to_load, base_folder, missing, crop_im = True):
    metal_folder = base_folder +"/" + channel_name_to_load
    image_path1 = image_filepath_for_3D_stack(metal_folder)
    image1 = imread(image_path1, pattern = None)
    
    if missing is not None:
        missing_slice_image = np.mean( np.array([image1[missing-1, :,:],image1[missing+1,:,:]]), axis=0)
        image1 =  np.insert(image1,missing, missing_slice_image, axis=0)
    
    for i in range(image1.shape[0]):
        #percent99 = np.percentile(image1[i, :,:], 99)
        #tmp_im = np.clip(image1[i, :,:],0,percent99)
        tmp_im = cv2.normalize(image1[i, :,:], None, alpha=0, beta=65535, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_16U)
        tmp_im = np.clip(tmp_im,0,65535)
        image1[i, :,:] = cv2.GaussianBlur(tmp_im,(3,3),1)
        #image1[i, :,:] = cv2.blur(tmp_im,(3,3))
               
    if crop_im == True:
         image1 = image1[:, y_start:y_end,x_start:x_end]
     
    print('Max pixel value:', np.max(image1))
    print('Median pixel value:', np.percentile(image1, 50))
    return image1

In [ ]:
# INPUT: single chanel TIFFs from the whole 3D model to use for 
#initial registeration i.e an image per slice
input_base = '~/LVI_breast_cancer/3D_registred_tiffs/IMC_fullStack_registred/imageJ_registration/full_model_aligned/'


labels_name = input_base +'measured_mask_final_segmentation_hwatershed_bg500_90%.tif'
stack_registred = input_base + 'SIMILARITY10_In115'

measurements = input_base + 'LVI_lymph_mean_intensities.csv'
lable_annotation = input_base + 'LVI_lymph_labels_area.csv'

results_file = input_base +'LVI_lymph_singleCell_analysis.h5ad'  # the file that will store the analysis results
cluster_labels_name = input_base + 'cluster_labels_image.tif'

#overlapping area of the image stack ie the area for the full 3D model used for downstream data analysis. 
row_start = 360 #y
row_end = 790   #y
col_start = 414 #x
col_end = 810 #x


In [ ]:
adata = sc.read_csv(measurements)
extra_annotation = pd.read_csv(lable_annotation)
adata.obs['cell_labels'] = list(extra_annotation['label'])
adata.obs['cell_labels'] = adata.obs['cell_labels'].astype('category')
adata.obs['cell_volume'] = list(extra_annotation['area'])
adata.obs['cell_volume'] = adata.obs['cell_volume'].astype('int64')
adata.write(results_file)

In [ ]:
adata = sc.read_h5ad(results_file)

In [ ]:
adata.var_names

In [ ]:
sc.set_figure_params(dpi=100, color_map = 'viridis_r')
sc.settings.verbosity = 1

In [ ]:
max_each_var = np.percentile(adata.X, 99, axis = 0)
min_each_var = np.min(adata.X,0)
range_var = max_each_var - min_each_var
adata.X = adata.X - min_each_var
adata.X = np.divide(adata.X, range_var)
adata.X[adata.X>1] = 1

sc.pp.neighbors(adata, n_neighbors=10,random_state = 111)
sc.tl.umap(adata,random_state = 111)
adata.write(results_file)

In [ ]:
communities, graph, Q = phenograph.cluster(adata.X, k=10, primary_metric= 'manhattan', seed = 10)
print(np.unique(communities))

In [ ]:
largest = max(np.unique(communities))+1
communities = [largest if x == 0 else x for x in communities]
adata.obs['phenograph'] = pd.Categorical(communities)

In [ ]:
adata.write(results_file)

In [ ]:
#from R color brewer
cluster_colors = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a',"#f0b98d", "#ef9708", "#0fcfc0",
                 '#8dd3c7','#ffffb3','#bebada','#fb8072','#80b1d3','#fdb462','#b3de69','#fccde5','#d9d9d9','#bc80bd','#ccebc5','#ffed6f',
                 '#8e0152','#c51b7d','#de77ae','#f1b6da','#fde0ef','#e6f5d0','#b8e186','#7fbc41','#4d9221','#276419']

In [ ]:
sc.pl.umap(adata,color=['phenograph'],palette=cluster_colors , save='_LVI_lymph_phenograph.pdf')

In [ ]:
sc.pl.umap(adata, color=['E/P Cadherin', 'GATA3', 'Her2', 'panCK', 'CD31'],save='_LVI_lymph_umap_tumor_markers.pdf')

In [ ]:
sc.pl.umap(adata, color=[ 'SMA', 'GLUT1', 'Vimentin'])

In [ ]:
sc.pl.umap(adata, color=['CD8a', 'CD4', 'CD3', 'CD20',  'CD45RO_RA'], save='_LVI_lymph_umap_immune_markers.pdf')

In [ ]:
rcParams['figure.figsize'] = 10,3
sc.pl.violin(adata, ['cell_volume'], groupby='phenograph', stripplot=False,  save = '_LVI_lymph_cell_volume_phenograph.pdf')  # use stripplot=False to remove the internal dots

In [ ]:
sc.pl.violin(adata, ['CD20', 'CD8a', 'CD4'], groupby='phenograph')

In [ ]:
sc.pl.violin(adata, ['CD3', 'CD45RO_RA', 'CD68'], groupby='phenograph')

In [ ]:
sc.pl.violin(adata, ['CD163', 'CD31', 'Vimentin'], groupby='phenograph')

In [ ]:
sc.pl.violin(adata, ['E/P Cadherin', 'Fibronectin', 'CD36'], groupby='phenograph')

In [ ]:
sc.pl.violin(adata, ['FSP1', 'S100A4', 'VEGF'], groupby='phenograph')

In [ ]:
sc.pl.violin(adata, ['GLUT1', 'CD44', 'CollagenI'], groupby='phenograph')

In [ ]:
sc.pl.violin(adata, ['Her2', 'TMEM173', 'panCK'], groupby='phenograph')

In [ ]:
sc.pl.violin(adata, ['cPARP_cCasp', 'GATA3', 'CD15'], groupby='phenograph')

In [ ]:
sc.pl.violin(adata, [ 'CD206', 'SMA'], groupby='phenograph')

In [ ]:
sc.pl.matrixplot(adata,adata.var_names, 'phenograph', dendrogram=True, cmap='Blues', standard_scale='var', 
                 colorbar_title='column scaled\nexpression', save = '_LVI_lymph_median_expression_phenograph.pdf')

In [ ]:
vp = sc.pl.stacked_violin(adata, var_names=adata.var_names, groupby= 'phenograph', colorbar_title='Median expression',dendrogram=True, standard_scale=None, stripplot=True, 
                          jitter=False, size=1,return_fig=True,ax=None)
vp.add_totals()

In [ ]:
vp.savefig('~/figures/stacked_violin_LVI_lymph_expression_phenograph.png')

##### Plot cluster analysis for 3D rendering

In [ ]:
cell_labels = imread(labels_name, pattern = None)
cell_labels  = np.squeeze(cell_labels)

In [ ]:
boundaries_only = np.zeros(cell_labels.shape, dtype = cell_labels.dtype)
k = 0 

while k < boundaries_only.shape[0]: 
    slice_2D = cell_labels[k, :,:]
    boundaries_only[k,:,:] = find_boundaries(slice_2D, connectivity=1, mode='outer', background=0)
    k  = k + 1
with_boundaries_mask =np.multiply(np.logical_not(boundaries_only), cell_labels)   

In [ ]:
cluster_labels = list(adata.obs['phenograph'])
object_labels = list(adata.obs['cell_labels'])


In [ ]:
cluster_labels_image =copy.deepcopy(with_boundaries_mask)


In [ ]:
for item in range(len(object_labels)):
    obi = int(object_labels[item])
    cluster_labels_image[with_boundaries_mask == obi] = int(cluster_labels[item])

In [ ]:
cluster_labels_image =cluster_labels_image.astype('uint16')
imsave(cluster_labels_name, cluster_labels_image)

In [ ]:
cluster_labels_image = imread(cluster_labels_name, pattern = None)
cluster_labels_image  = np.squeeze(cluster_labels_image)

In [ ]:
cluster_colors_napari = copy.deepcopy(cluster_colors)
cluster_colors_napari.insert(0,'#000000')
cmap = Colormap(transform_color(cluster_colors_napari))

In [ ]:
#assumes that colors in the color_map are ordered according to ascending order of cluter numbers
napari_color_dict = {}
i=0
while i < len(cluster_colors_napari):
    napari_color_dict[i] = cluster_colors_napari[i]
    i +=1

In [ ]:
colors = {
            label: transform_color(color_str)[0]
            for label, color_str in napari_color_dict.items()
        }


In [ ]:
scale_factors = [2,1,1]

In [ ]:
channel_name_to_load ='Sm154'
met_folder = stack_registred +"/" + channel_name_to_load
image_path1 = image_filepath_for_3D_stack(met_folder)
image1 = imread(image_path1, pattern = None)
image1 = np.squeeze(image1)
image1 = image1[:,row_start:row_end,col_start:col_end]
im_name = input_base + 'cd15_ROI_image.tif'
imsave(im_name,image1) 

In [ ]:
image1.shape

In [ ]:
channel_1= 'Ir191'
stack1 = load_channel_stack_for_napari(channel_1,stack_registred, None,False )
channel_2= 'GLUT1'
stack2 = load_channel_stack_for_napari('Pr141',stack_registred, None,False )
channel_3= 'SMA'
stack3 = load_channel_stack_for_napari('In115',stack_registred, None,False )
channel_4= 'Vimentin'
stack4 = load_channel_stack_for_napari('Sm149',stack_registred, None,False )




In [ ]:
with napari.gui_qt():
    viewer = napari.view_image(stack1[:,row_start:row_end,col_start:col_end], name = channel_1, scale = scale_factors)
    viewer.add_image(stack2[:,row_start:row_end,col_start:col_end], name = channel_2, scale = scale_factors)
    viewer.add_image(stack3[:,row_start:row_end,col_start:col_end], name = channel_3, scale = scale_factors)
    viewer.add_image(stack4[:,row_start:row_end,col_start:col_end], name = channel_4, scale = scale_factors)
    viewer.add_labels(cluster_labels_image, name='clusters', color = colors, scale = scale_factors)

##### Vizualise specific clusters of interest

In [ ]:
cluster_labels_image_subset = copy.deepcopy(cluster_labels_image)
unique_clusters = list(np.unique(cluster_labels_image_subset))

In [ ]:
clusters_to_display = [19,7,14,22]

In [ ]:
for i in unique_clusters:
    if i not in clusters_to_display:
        cluster_labels_image_subset[cluster_labels_image_subset==i] = 0
image_name = input_base + 'cluster6_7_12_19_tmem_ecahderin_cluster_labels_image.tif'
imsave(image_name, cluster_labels_image_subset)

In [ ]:
with napari.gui_qt():
    viewer = napari.view_image(stack1[:,row_start:row_end,col_start:col_end], name = channel_1, scale = scale_factors)    
    viewer.add_image(stack2[:,row_start:row_end,col_start:col_end], name = channel_2, scale = scale_factors)
    viewer.add_image(cluster_labels_image_subset, name='subset of clusters', scale = scale_factors)
    viewer.add_image(stack4[:,row_start:row_end,col_start:col_end], name = channel_4, scale = scale_factors)
    viewer.add_labels(cluster_labels_image, name='clusters', color = colors, scale = scale_factors)

##### Display expression values of a select marker of interest on cells

In [ ]:
cell_labels_select_markers = copy.deepcopy(with_boundaries_mask)
cell_labels_select_markers =cell_labels_select_markers.astype('int32')

marker_of_interest = 'TMEM173'
dict_mean_SMA = {}

for x in range(len(adata.obs['cell_labels'])):
    object_label = adata.obs['cell_labels'][x]
    a_marker = adata[adata.obs['cell_labels']== object_label, marker_of_interest]
    dict_mean_SMA[object_label] = float(a_marker.X)

mean_marker_image = np.zeros(cell_labels_select_markers.shape)
for key in dict_mean_SMA.keys():
    mean_marker_image[cell_labels_select_markers == key] = round(dict_mean_SMA[key],3)

cell_mean_select_markers_name = input_base + 'TMEM173_mean_labels_image.tif'
imsave(cell_mean_select_markers_name,mean_marker_image) 

In [ ]:
with napari.gui_qt():
    viewer = napari.view_image(cell_labels_select_markers, name = marker_of_interest, scale = scale_factors)

End of Notebook